In [1]:
import pymysql
import pandas as pd
import numpy as np

### 데이터베이스(MySQL) 연결

In [2]:
try:
    # MySQL 데이터베이스에 연결
    conn = pymysql.connect(
        host='capstone-db.cfgmik22w69x.ap-northeast-2.rds.amazonaws.com',
        user='root',
        password='daejin2019',
        db='capstone',
        charset='utf8',
        port=3306
    )
    
    if conn.open:  # is_connected() 대신 open 속성을 사용
        print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")

except pymysql.MySQLError as e:
    print(f"연결 중 오류 발생: {e}")

# finally:
#     if 'connection' in locals() and connection.open:
#         connection.close()
#         print("MySQL 데이터베이스와의 연결이 종료되었습니다.")


MySQL 데이터베이스에 성공적으로 연결되었습니다.


In [3]:
cursor = conn.cursor()

In [4]:
# Load the uploaded CSV files
file_path = '../data/hotel_after.csv'
room_file_path = '../data/room.csv'

lodgings_data = pd.read_csv(file_path)
rooms_data = pd.read_csv(room_file_path)
lodgings_data.head(5)

,contentid,title,class,area,sigungu,addr,tel,firstimage,firstimage2
0,2465071,가경재,한옥,경상북도,안동시,경상북도 안동시 하회남촌길 69-5,054-855-8552,http://tong.visitkorea.or.kr/cms/resource/00/2...,http://tong.visitkorea.or.kr/cms/resource/00/2...
1,142785,가락관광호텔,관광호텔,서울특별시,송파구,서울특별시 송파구 송파대로28길 5,02-400-6641~3,NaN,NaN
2,2671267,가락청,한옥,전북특별자치도,전주시,전북특별자치도 전주시 완산구 한지길 68,NaN,NaN,NaN
3,2627867,가람나무,한옥,경기도,파주시,경기도 파주시 소라지로327번길 126-21,NaN,NaN,NaN
4,1865597,가람초연재,한옥,경상북도,안동시,경상북도 안동시 풍천면 하회종가길 76-6,010-3849-7542,http://tong.visitkorea.or.kr/cms/resource/48/2...,http://tong.visitkorea.or.kr/cms/resource/48/2...


In [5]:
rooms_data.head(5)

,contentid,roomcode,roomtitle,roomcount,available_count,base_person,max_person,offseasonminfee,offseasonmaxfee,peakseasonminfee,...,roomsofa,roomcook,roomtable,roomhairdryer,roomimg1,roomimg1_alt,roomimg2,roomimg2_alt,roomimg3,roomimg3_alt
0,2465071,46591,사랑채 아랫방,6,5,2,2,80000,122000,127000,...,NaN,NaN,Y,Y,http://tong.visitkorea.or.kr/cms/resource/16/2...,경북_안동_가경재_사랑채(아랫방)03,http://tong.visitkorea.or.kr/cms/resource/15/2...,경북_안동_가경재_사랑채(아랫방)02,http://tong.visitkorea.or.kr/cms/resource/14/2...,경북_안동_가경재_사랑채(아랫방)01
1,2465071,46593,군불 황토 아랫방,6,5,3,7,100000,141000,108000,...,NaN,NaN,Y,Y,http://tong.visitkorea.or.kr/cms/resource/23/2...,경북_안동_가경재_군불황토(아랫방)05,http://tong.visitkorea.or.kr/cms/resource/22/2...,경북_안동_가경재_군불황토(아랫방)04,http://tong.visitkorea.or.kr/cms/resource/19/2...,경북_안동_가경재_군불황토(아랫방)01
2,142785,11432,스탠다드 더블,8,2,2,3,98000,120000,109000,...,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
3,142785,11433,스탠다드 트윈,8,6,2,4,108000,137000,155000,...,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN
4,142785,43642,디럭스 더블,14,5,2,5,108000,147000,133000,...,Y,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Lodgings 테이블에 데이터 추가
# Prepare and execute SQL insert statements for the Lodgings table
lodging_id_map = {}  # Map to store contentid to lodging_id mapping
for _, row in lodgings_data.iterrows():
    name = row['title'].replace("'", "''") if pd.notnull(row['title']) else None
    part = row['class'].replace("'", "''") if pd.notnull(row['class']) else None
    area = row['area'].replace("'", "''") if pd.notnull(row['area']) else None
    sigungu = row['sigungu'].replace("'", "''") if pd.notnull(row['sigungu']) else None
    address = row['addr'].replace("'", "''") if pd.notnull(row['addr']) else None
    tel = row['tel'].replace("'", "''") if pd.notnull(row['tel']) else None
    main_image = row['firstimage'].replace("'", "''") if pd.notnull(row['firstimage']) else None

    insert_statement = (
        "INSERT INTO Lodgings (name, part, area, sigungu, address, tel, main_image) "
        "VALUES (%s, %s, %s, %s, %s, %s, %s)"
    )
    cursor.execute(insert_statement, (name, part, area, sigungu, address, tel, main_image))
    lodging_id = cursor.lastrowid  # Get the auto-incremented lodging_id
    lodging_id_map[row['contentid']] = lodging_id

In [9]:
conn.commit()

In [10]:
# Rooms 테이블에 데이터 추가
# Prepare and execute SQL insert statements for the Rooms table
room_id_map = {}  # Map to store roomcode to room_id mapping
for _, row in rooms_data.iterrows():
    contentid = row['contentid']
    lodging_id = lodging_id_map.get(contentid)  # Get the corresponding lodging_id from the map
    room_name = row['roomtitle'].replace("'", "''") if pd.notnull(row['roomtitle']) else None
    price_per_night = row['offseasonminfee'] if pd.notnull(row['offseasonminfee']) else None
    min_occupancy = row['base_person'] if pd.notnull(row['base_person']) else None
    max_occupancy = row['max_person'] if pd.notnull(row['max_person']) else None
    room_count = row['roomcount'] if pd.notnull(row['roomcount']) else None

    insert_statement = (
        "INSERT INTO Rooms (lodging_id, room_name, price_per_night, min_occupancy, max_occupancy, room_count) "
        "VALUES (%s, %s, %s, %s, %s, %s)"
    )
    cursor.execute(insert_statement, (lodging_id, room_name, price_per_night, min_occupancy, max_occupancy, room_count))
    room_id = cursor.lastrowid  # Get the auto-incremented room_id
    room_id_map[row['roomcode']] = room_id

In [11]:
conn.commit()

In [14]:
# Facilities 테이블에 데이터 추가
for _, row in rooms_data.iterrows():
    room_id = room_id_map.get(row['roomcode'])  # Get the corresponding room_id from the map
    has_bathfacility = row['roombathfacility'] if pd.notnull(row['roombathfacility']) else None
    has_bath = row['roombath'].replace("'", "''") if pd.notnull(row['roombath']) else None
    has_home_theater = row['roomhometheater'] if pd.notnull(row['roomhometheater']) else None
    has_air_conditioning = row['roomaircondition'] if pd.notnull(row['roomaircondition']) else None
    has_tv = row['roomtv'] if pd.notnull(row['roomtv']) else None
    has_pc = row['roompc'] if pd.notnull(row['roompc']) else None
    cable = row['roomcable'] if pd.notnull(row['roomcable']) else None
    has_internet = row['roominternet'] if pd.notnull(row['roominternet']) else None
    has_refrigerator = row['roomrefrigerator'] if pd.notnull(row['roomrefrigerator']) else None
    has_toiletries = row['roomtoiletries'] if pd.notnull(row['roomtoiletries']) else None
    has_sofa = row['roomsofa'] if pd.notnull(row['roomsofa']) else None
    has_cook = row['roomcook'] if pd.notnull(row['roomcook']) else None
    has_table = row['roomtable'] if pd.notnull(row['roomtable']) else None
    hairdryer = row['roomhairdryer'] if pd.notnull(row['roomhairdryer']) else None

    insert_statement = (
        "INSERT INTO Facilities (room_id, has_bathfacility, has_bath, has_home_theater, has_air_conditioning, has_tv, has_pc, cable, "
        "has_internet, has_refrigerator, has_toiletries, has_sofa, has_cook, has_table, hairdryer) "
        "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    )
    cursor.execute(insert_statement, (room_id, has_bathfacility, has_bath, has_home_theater, has_air_conditioning, has_tv, has_pc,
                                       cable, has_internet, has_refrigerator, has_toiletries, has_sofa, has_cook, has_table, hairdryer))


In [15]:
conn.commit()

In [16]:
# Rooms_photo 테이블에 데이터 추가
for _, row in rooms_data.iterrows():
    room_id = room_id_map.get(row['roomcode'])  # Get the corresponding room_id from the map
    photo_urls = [row['roomimg1'], row['roomimg2'], row['roomimg3']]  # Get the photo URLs

    for photo_url in photo_urls:
        if pd.notnull(photo_url):
            insert_statement = (
                "INSERT INTO Room_photos (room_id, photo_url) "
                "VALUES (%s, %s)"
            )
            cursor.execute(insert_statement, (room_id, photo_url))


In [17]:
# 저장, 연결 끊기
conn.commit()

In [18]:
cursor.close()
conn.close()